<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-final-PG-NO-147/blob/main/aimlvgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from IPython.display import clear_output
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'supuntheekshanam'
os.environ['KAGGLE_KEY'] = '97fb6c99381c9d6779d3f904ef279dce'
clear_output()

In [31]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [32]:
!unzip /content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip
clear_output()

In [4]:
import tensorflow as tf
import os
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

training_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

print(class_names)

Found 48001 files belonging to 26 classes.
['Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Cherry_(including_sour)___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Apple___Apple_scab', 'Tomato___Septoria_leaf_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Apple___Black_rot', 'Squash___Powdery_mildew', 'Tomato___Early_blight', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Leaf_Mold', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Grape___Black_rot', 'Tomato___Bacterial_spot', 'Corn_(maize)___Northern_Leaf_Blight', 'Potato___Late_blight', 'Potato___Early_blight', 'Apple___Cedar_apple_rust', 'Peach___Bacterial_spot', 'Orange___Haunglongbing_(Citrus_greening)', 'Grape___Esca_(Black_Measles)']


In [5]:
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

validation_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

Found 12000 files belonging to 26 classes.


In [6]:
print("Classes used for training:")
print(class_names)

Classes used for training:
['Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Cherry_(including_sour)___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Apple___Apple_scab', 'Tomato___Septoria_leaf_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Apple___Black_rot', 'Squash___Powdery_mildew', 'Tomato___Early_blight', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Leaf_Mold', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Grape___Black_rot', 'Tomato___Bacterial_spot', 'Corn_(maize)___Northern_Leaf_Blight', 'Potato___Late_blight', 'Potato___Early_blight', 'Apple___Cedar_apple_rust', 'Peach___Bacterial_spot', 'Orange___Haunglongbing_(Citrus_greening)', 'Grape___Esca_(Black_Measles)']


In [7]:
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))
validation_set = validation_set.map(lambda x, y: (Rescaling(1./255)(x), y))

In [8]:
from tensorflow.keras import layers
geometric_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

def apply_geometric_augmentation(image, label):
    image = geometric_augmentation(image)
    return image, label

training_set = training_set.map(apply_geometric_augmentation)

In [15]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model (without the top classification layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base (so pretrained weights don’t get modified)
for layer in base_model.layers:
    layer.trainable = False

# Build your custom model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')  # Adjust number of classes
])

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(training_set, validation_data=validation_set, epochs=10)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1063s 699ms/step - accuracy: 0.4864 - loss: 1.7757 - val_accuracy: 0.8103 - val_loss: 0.6243
Epoch 2/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1024s 682ms/step - accuracy: 0.7543 - loss: 0.7828 - val_accuracy: 0.8567 - val_loss: 0.4634
Epoch 3/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1044s 696ms/step - accuracy: 0.7858 - loss: 0.6744 - val_accuracy: 0.8574 - val_loss: 0.4490
Epoch 4/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1037s 691ms/step - accuracy: 0.8086 - loss: 0.5877 - val_accuracy: 0.8727 - val_loss: 0.3956
Epoch 5/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1034s 689ms/step - accuracy: 0.8313 - loss: 0.5170 - val_accuracy: 0.8753 - val_loss: 0.3851
Epoch 6/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1027s 684ms/step - accuracy: 0.8308 - loss: 0.5075 - val_accuracy: 0.8770 - val_loss: 0.3657
Epoch 7/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1029s 685ms/step - accuracy: 0.8512 - loss: 0.4580 - val_accuracy: 0.8889 - val_loss: 0.3303
Epoc

In [16]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,291,408 (203.29 MB)

 Trainable params: 12,858,906 (49.05 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 25,717,814 (98.11 MB)

In [17]:
import sys
from io import StringIO

stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)

with open("/content/drive/MyDrive/model_summary.txt", "w") as f:
    f.write(short_model_summary)

print("Model summary saved to /content/drive/MyDrive/models/model_summary.txt")

Model summary saved to /content/drive/MyDrive/models/model_summary.txt


In [18]:
train_loss,train_acc = model.evaluate(training_set)

1501/1501 ━━━━━━━━━━━━━━━━━━━━ 961s 640ms/step - accuracy: 0.9131 - loss: 0.2671


In [19]:
val_loss,val_acc = model.evaluate(validation_set)

375/375 ━━━━━━━━━━━━━━━━━━━━ 64s 170ms/step - accuracy: 0.8824 - loss: 0.3500


In [20]:
model.save("/content/drive/MyDrive/models/modelVgg16.keras")

In [21]:
model = tf.keras.models.load_model("/content/drive/MyDrive/models/modelVgg16.keras")

In [26]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/test/test/TomatoYellowCurlVirus3.JPG'

img = image.load_img(img_path, target_size=(224, 224))

img_array = image.img_to_array(img)


img_array = np.expand_dims(img_array, axis=0)

preprocessed_image = img_array / 255.0

In [27]:
prediction = model.predict(preprocessed_image)
prediction,prediction.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


(array([[1.2600332e-01, 7.7713457e-07, 5.1433060e-05, 5.8166683e-05,
         1.7261925e-09, 1.6404200e-02, 5.0623668e-03, 2.6566726e-05,
         1.4518809e-03, 1.0379241e-07, 4.3102230e-07, 1.9120227e-05,
         3.6541041e-02, 6.9953196e-02, 8.4080726e-02, 2.0393352e-04,
         6.5829831e-01, 3.2462669e-06, 4.4068194e-04, 6.5084323e-06,
         1.3182613e-03, 4.7315837e-07, 1.4482617e-05, 2.9656337e-06,
         2.2711449e-06, 5.5497323e-05]], dtype=float32),
 (1, 26))

In [28]:
result_index = np.argmax(prediction)
result_index

np.int64(16)

In [29]:
class_names[result_index]

'Tomato___Tomato_mosaic_virus'